In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [3]:
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices) <= 0:
  raise ValueError("No GPU detected!")
print("Num GPUs:", len(physical_devices))

Num GPUs: 1


In [4]:
classes = ["ant", "fish", "tree", "bee", "sun", "star", "clock", "flower", "car", "cactus"]
print(len(classes))

10


In [0]:
import urllib.request
def download(classes):
  
  base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
  for c in classes:
    cls_url = c.replace('_', '%20')
    path = base+cls_url+'.npy'
    print(path)
    urllib.request.urlretrieve(path, c+'.npy')

In [6]:
download(classes)

https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/ant.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/fish.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/tree.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bee.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/sun.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/star.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/clock.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/flower.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/car.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/cactus.npy


In [7]:
import os
import glob
import numpy as np

all_files = glob.glob('*.npy')

features = np.empty([0, 784])
labels = np.empty([0])
class_names = []

print(all_files)
data_per_class = 50000

for i, file in enumerate(all_files):
  data = np.load(file)
  data = data[0: data_per_class, :]
  labels_array = np.full(data.shape[0], i)

  features = np.concatenate((features, data), axis=0)
  labels = np.append(labels, labels_array)

  class_name, ext = os.path.splitext(os.path.basename(file))
  class_names.append(class_name)

data = None
labels_array = None

print(class_names)

['car.npy', 'tree.npy', 'flower.npy', 'fish.npy', 'ant.npy', 'clock.npy', 'bee.npy', 'sun.npy', 'star.npy', 'cactus.npy']
['car', 'tree', 'flower', 'fish', 'ant', 'clock', 'bee', 'sun', 'star', 'cactus']


In [0]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [9]:
features, labels = shuffle(features, labels)

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=0, test_size=0.1)

train_features = train_features.reshape(train_features.shape[0], 28, 28, 1).astype('float32')
test_features = test_features.reshape(test_features.shape[0], 28, 28, 1).astype('float32')

train_features = train_features / 255.0
test_features = test_features / 255.0


train_labels = keras.utils.to_categorical(train_labels, 10)
test_labels = keras.utils.to_categorical(test_labels, 10)

print(train_features.shape)

(450000, 28, 28, 1)


In [0]:
model = keras.models.Sequential([
                                  keras.layers.Conv2D(32, (5,5), input_shape=(28, 28, 1), activation='relu'),
                                  keras.layers.Conv2D(32, (5, 5), activation='relu'),
                                  keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
                                  keras.layers.Dropout(0.25),
                                  
                                  keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
                                  keras.layers.Conv2D(64, (5, 5), activation='relu'),
                                  keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),

                                  keras.layers.Flatten(),
                                  keras.layers.Dense(512, activation='relu'),
                                  keras.layers.Dropout(0.5),
                                  keras.layers.Dense(128, activation='relu'),
                                  keras.layers.Dropout(0.5),
                                  keras.layers.Dense(10, activation='softmax'),

])

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [12]:
model.fit(train_features, train_labels, validation_data=(test_features, test_labels), epochs=50, batch_size=64)

Epoch 1/50
7032/7032 [==============================] - 27s 4ms/step - loss: 0.3436 - accuracy: 0.8982 - val_loss: 0.1962 - val_accuracy: 0.9408
Epoch 2/50
7032/7032 [==============================] - 27s 4ms/step - loss: 0.2207 - accuracy: 0.9363 - val_loss: 0.1773 - val_accuracy: 0.9474
Epoch 3/50
7032/7032 [==============================] - 27s 4ms/step - loss: 0.1960 - accuracy: 0.9435 - val_loss: 0.1762 - val_accuracy: 0.9483
Epoch 4/50
7032/7032 [==============================] - 27s 4ms/step - loss: 0.1834 - accuracy: 0.9469 - val_loss: 0.1598 - val_accuracy: 0.9523
Epoch 5/50
7032/7032 [==============================] - 27s 4ms/step - loss: 0.1750 - accuracy: 0.9497 - val_loss: 0.1511 - val_accuracy: 0.9541
Epoch 6/50
7032/7032 [==============================] - 26s 4ms/step - loss: 0.1686 - accuracy: 0.9515 - val_loss: 0.1518 - val_accuracy: 0.9550
Epoch 7/50
7032/7032 [==============================] - 27s 4ms/step - loss: 0.1638 - accuracy: 0.9525 - val_loss: 0.1510 - val_ac

In [13]:
loss, accuracy = model.evaluate(test_features, test_labels)
print("loss: {}, accuracy: {}".format(loss, accuracy))

1563/1563 [==============================] - 4s 2ms/step - loss: 0.1533 - accuracy: 0.9585
loss: 0.15332891047000885, accuracy: 0.9584800004959106


In [0]:
model.save("model.h5")

In [0]:
with open("classes.txt", mode="w") as f:
  for class_name in class_names:
    f.write(class_name)
    f.write("\n")